# Networks

This project will investigate the degree distribution produced by a growing network model, undirected [Price](https://en.wikipedia.org/wiki/Price%27s_model)/[Barabasi and Albert model](https://en.wikipedia.org/wiki/Barab%C3%A1si%E2%80%93Albert_model). The degree distribution will follow a kind of [fat tailed distribution](https://en.wikipedia.org/wiki/Fat-tailed_distribution) called [Yule-Simon distribution](https://en.wikipedia.org/wiki/Yule%E2%80%93Simon_distribution).\
The algorithms used is shown as below:
![Rules for BA (Price) model](.\RefferedImagesByIpynb\1.png)

At time $t_{0}$, each node $ i $ have degree $k_{i}$ and thus the probability of been connected to a new node at $t+1$ is $\sqcap(k,t) = \frac{k_{i}}{\sum_{i}k_{i}} = \frac{k_{i}}{2E(t)}$.  

So, we can get the ***Master Equation*** $$ n(k, t+1) = n(k,t) + m\sqcap(k-1,t)n(k-1,t) - m\sqcap(k,t)n(k,t) + \delta_{k,m}$$
The long time and large $k$ solution will be $$\lim_{t\to\infty}p(k,t) = p_{\infty}(k) \propto \frac{1}{k^3}$$This mathematical model can be verified by numerical simulation.

The base of simulation would be an implementation of graph data structure. This is perfered to be an adjacency list for overal good time and space complexity and easy to be implemented and manipulated. I will just use `networkx` lib for python. The library used adjacency map (a.k.a `dict` in python). Other required modules are `numpy`, `scipy`, `matplotlib` and `bokeh` (`bokeh` because it create better plots in ipynb). The environment I have is `anaconda version=2020.11 build=py38_0`. The data structure employed by `networkx` is adjacency map, this might because python list cost too much memory.

In [1]:
import networkx as nx
import random

## Phase 1: Preferential attachment

In [2]:
def pref(N, m = 3, n = 3, seed = 10):
    """This function generate a BA graph of N nodes with each added node have m
    edges.
    
    Parameters
    ----------
    N: total number of nodes
    m: number of edges attached to each new node, or degree of new node.
       random: a random number generator to satisfy the stochastic process.
    n: number of nodes for initial graph, when n=m+1, this methods equals to
       pure random attachment method
    seed: the random seed
    
    Returns
    -------
    A Barabasi Albert Graph object created by nexworkx, edges of new nodes
    determined by preferential attachment.
    """
    # first make sure the edges of new node is an legal option
    if m >= N:
        raise Exception("too many edges, greater than N")
    if m < 1:
        raise Exception("at least 1 edge for new node")
    if n < m: 
        raise Exception("at least m node in initial graph")
        
    random.seed(seed) 
        
    # Creates new graph of n nodes, of equal degree
    nodes = list(range(n))
    G = nx.complete_graph(nodes)
    G.name = "PrefGraph with N = %s, m = %s"%(N,m)
    
    # nodes list with occurance probability proportional to k
    # the list is maintained for random sampling
    nodes_list = []
    for i in nodes:
        nodes_list.extend([i]*(n-1))  # initial degree are n-1
    
    target = set()
    
    # Target nodes for new edges
    while len(target) < m:
        target.add(random.choice(nodes_list))
    
    total = n
    
    while total < N:
        new_stubs = [total]*m
        new_edges = zip(new_stubs,target)
        G.add_edges_from(new_edges)
        
        #add new edges to the list
        nodes_list.extend(new_stubs)
        nodes_list.extend(list(target))
        
        # m nodes are chosen from the edge_list to form new targets.
        target = set()
        while len(target)< m:
            random_node =random.choice(nodes_list)
            target.add(random_node)
        total += 1
        print(G.adj)
        print('*********************')
        print(G.degree)
        print('=====================')
    return G

# an example
G = pref(10,3,3)

{0: {1: {}, 2: {}, 3: {}}, 1: {0: {}, 2: {}, 3: {}}, 2: {0: {}, 1: {}, 3: {}}, 3: {0: {}, 1: {}, 2: {}}}
*********************
[(0, 3), (1, 3), (2, 3), (3, 3)]
{0: {1: {}, 2: {}, 3: {}, 4: {}}, 1: {0: {}, 2: {}, 3: {}, 4: {}}, 2: {0: {}, 1: {}, 3: {}}, 3: {0: {}, 1: {}, 2: {}, 4: {}}, 4: {0: {}, 1: {}, 3: {}}}
*********************
[(0, 4), (1, 4), (2, 3), (3, 4), (4, 3)]
{0: {1: {}, 2: {}, 3: {}, 4: {}, 5: {}}, 1: {0: {}, 2: {}, 3: {}, 4: {}}, 2: {0: {}, 1: {}, 3: {}}, 3: {0: {}, 1: {}, 2: {}, 4: {}, 5: {}}, 4: {0: {}, 1: {}, 3: {}, 5: {}}, 5: {0: {}, 3: {}, 4: {}}}
*********************
[(0, 5), (1, 4), (2, 3), (3, 5), (4, 4), (5, 3)]
{0: {1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}}, 1: {0: {}, 2: {}, 3: {}, 4: {}}, 2: {0: {}, 1: {}, 3: {}, 6: {}}, 3: {0: {}, 1: {}, 2: {}, 4: {}, 5: {}}, 4: {0: {}, 1: {}, 3: {}, 5: {}}, 5: {0: {}, 3: {}, 4: {}, 6: {}}, 6: {0: {}, 2: {}, 5: {}}}
*********************
[(0, 6), (1, 4), (2, 4), (3, 5), (4, 4), (5, 4), (6, 3)]
{0: {1: {}, 2: {}, 3: {}, 4: {

With the help of this implementation, we can start to analyse graphs.

##  Phase 2: random attachement

In [3]:
def ran(N, m=3, n = 3, seed = 10):
    """This function generate a BA graph of N nodes with each added node have m
    edges.
    
    Parameters
    ----------
    N: total number of nodes
    m: number of edges attached to each new node, or degree of new node.
       random: a random number generator to satisfy the stochastic process.
    n: number of nodes for initial graph, when n=m+1, this methods equals to
       pure random attachment method
    seed: the random seed
    
    Returns
    -------
    A Barabasi Albert Graph object created by nexworkx, edges of new nodes
    determined by preferential attachment.
    """
    # first make sure the edges of new node is an legal option
    if m >= N:
        raise Exception("too many edges, greater than N")
    if m < 1:
        raise Exception("at least 1 edge for new node")
    if n < m: 
        raise Exception("at least m node in initial graph")
        
    random.seed(seed)
        
    # Creates new graph of n nodes, and no edges
    G = nx.generators.classic.empty_graph(n)
    G.name = "RandGraph with N = %s, m = %s"%(N,n)
    
    # maintain a nodes_list to do the random choice
    # for preferential attachement, node i appear k_i times
    nodes_list = list(range(n))
    
    total = n
    
    # generate the whole graph
    while total <= N:
        # the new node equals to the current number of nodes
        new_node = total
        G.add_node(total)
        # new edges added by pure random sampling
        random_nodes = random.sample(nodes_list, m)
        for i in random_nodes:
            G.add_edge(new_node, i)
        # update the nodes list after adding edge
        nodes_list.append(new_node)
        total += 1
        print(G.adj)
        print('*********************')
        print(G.degree)
        print('=====================')
    return G

# an example
G = ran(10,3,3)

{0: {3: {}}, 1: {3: {}}, 2: {3: {}}, 3: {2: {}, 0: {}, 1: {}}}
*********************
[(0, 1), (1, 1), (2, 1), (3, 3)]
{0: {3: {}, 4: {}}, 1: {3: {}}, 2: {3: {}, 4: {}}, 3: {2: {}, 0: {}, 1: {}, 4: {}}, 4: {3: {}, 2: {}, 0: {}}}
*********************
[(0, 2), (1, 1), (2, 2), (3, 4), (4, 3)]
{0: {3: {}, 4: {}}, 1: {3: {}, 5: {}}, 2: {3: {}, 4: {}}, 3: {2: {}, 0: {}, 1: {}, 4: {}, 5: {}}, 4: {3: {}, 2: {}, 0: {}, 5: {}}, 5: {1: {}, 3: {}, 4: {}}}
*********************
[(0, 2), (1, 2), (2, 2), (3, 5), (4, 4), (5, 3)]
{0: {3: {}, 4: {}, 6: {}}, 1: {3: {}, 5: {}, 6: {}}, 2: {3: {}, 4: {}, 6: {}}, 3: {2: {}, 0: {}, 1: {}, 4: {}, 5: {}}, 4: {3: {}, 2: {}, 0: {}, 5: {}}, 5: {1: {}, 3: {}, 4: {}}, 6: {2: {}, 1: {}, 0: {}}}
*********************
[(0, 3), (1, 3), (2, 3), (3, 5), (4, 4), (5, 3), (6, 3)]
{0: {3: {}, 4: {}, 6: {}}, 1: {3: {}, 5: {}, 6: {}}, 2: {3: {}, 4: {}, 6: {}, 7: {}}, 3: {2: {}, 0: {}, 1: {}, 4: {}, 5: {}, 7: {}}, 4: {3: {}, 2: {}, 0: {}, 5: {}, 7: {}}, 5: {1: {}, 3: {}, 4: {}},

## Phase 3: Mixed Preferential and Random Attachment

In [4]:
def mix(N, m=3, n = 3, q = 0.5, seed = 10):
    """This function generate a BA graph of N nodes with each added node have m
    edges.
    
    Parameters
    ----------
    N: total number of nodes
    m: number of edges attached to each new node, or degree of new node.
       random: a random number generator to satisfy the stochastic process.
    n: number of nodes for initial graph, when n=m+1, this methods equals to
       pure random attachment method
    seed: the random seed
    q: the probability that one edge is determined by preferential
    
    Returns
    -------
    A Barabasi Albert Graph object created by nexworkx, edges of new nodes
    determined by preferential attachment.
    """
    # first make sure the edges of new node is an legal option
    if m >= N:
        raise Exception("too many edges, greater than N")
    if m < 1:
        raise Exception("at least 1 edge for new node")
    if n < m: 
        raise Exception("at least m node in initial graph")
        
    random.seed(seed)
        
    # Creates new graph of n nodes, of equal degree
    nodes = list(range(n))
    G = nx.complete_graph(nodes)
    G.name = "MixGraph with N = %s, m = %s"%(N,m)
    
    # nodes list with occurance probability proportional to k
    # the list is maintained for random sampling by preferential
    nodes_list = []
    for i in nodes:
        nodes_list.extend([i]*(n-1))  # initial degree are n-1
    #nodes set for all nodes
    nodes_set = set(range(n))
    
    total = n
    
    while total < N:
        new_stubs = [total]*m
        random_nodes = set()
        
        while len(random_nodes) < m:
            if random.random() < q:
                # preferential
                random_node = random.choice(nodes_list)
                while random_node in random_nodes:
                    random_node = random.choice(nodes_list)
                random_nodes.add(random_node)
            else:
                random_node = random.choice(list(nodes_set))
                while random_node in random_nodes:
                    random_node = random.choice(list(nodes_set))
                random_nodes.add(random_node)
            
        new_edges = zip(new_stubs, random_nodes)
        G.add_edges_from(new_edges)
        
        #add new edges to the list and update nodes set
        nodes_list.extend(new_stubs)
        nodes_list.extend(list(random_nodes))
        nodes_set.add(total)
        
        total += 1
        print(G.adj)
        print('*********************')
        print(G.degree)
        print('=====================')
    return G
    
G = mix(10,3,3)

{0: {1: {}, 2: {}, 3: {}}, 1: {0: {}, 2: {}, 3: {}}, 2: {0: {}, 1: {}, 3: {}}, 3: {0: {}, 1: {}, 2: {}}}
*********************
[(0, 3), (1, 3), (2, 3), (3, 3)]
{0: {1: {}, 2: {}, 3: {}, 4: {}}, 1: {0: {}, 2: {}, 3: {}}, 2: {0: {}, 1: {}, 3: {}, 4: {}}, 3: {0: {}, 1: {}, 2: {}, 4: {}}, 4: {0: {}, 2: {}, 3: {}}}
*********************
[(0, 4), (1, 3), (2, 4), (3, 4), (4, 3)]
{0: {1: {}, 2: {}, 3: {}, 4: {}, 5: {}}, 1: {0: {}, 2: {}, 3: {}}, 2: {0: {}, 1: {}, 3: {}, 4: {}, 5: {}}, 3: {0: {}, 1: {}, 2: {}, 4: {}}, 4: {0: {}, 2: {}, 3: {}, 5: {}}, 5: {0: {}, 2: {}, 4: {}}}
*********************
[(0, 5), (1, 3), (2, 5), (3, 4), (4, 4), (5, 3)]
{0: {1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}}, 1: {0: {}, 2: {}, 3: {}}, 2: {0: {}, 1: {}, 3: {}, 4: {}, 5: {}, 6: {}}, 3: {0: {}, 1: {}, 2: {}, 4: {}, 6: {}}, 4: {0: {}, 2: {}, 3: {}, 5: {}}, 5: {0: {}, 2: {}, 4: {}}, 6: {0: {}, 2: {}, 3: {}}}
*********************
[(0, 6), (1, 3), (2, 6), (3, 5), (4, 4), (5, 3), (6, 3)]
{0: {1: {}, 2: {}, 3: {}, 4: {

## Now we can do the data acqurization

```python
from implementation import *
  # this is the implementation modified from the implementation above
  # the functions generate adjacency list file for each graph

# N = [1_000_000] * 7 # 1 million should be ok
# 1 million will use at least 10 Gib of memory for m = 64
# got memory error, so use smaller N
N = [500_000] * 7
m = [2, 4, 8, 16, 32, 64, 128]
q = [0, 1/6, 2/6, 3/6, 4/6, 5/6, 1]

pref_arg = list(zip(N, m, m))
ran_arg = list(zip(N, m, m))
mix_arg = list(zip(N, m, m, q))

from multiprocessing import Pool

with Pool(7) as p:
    p.starmap(pref, pref_arg)
    p.join()
    p.starmap(ran, ran_arg)
    p.join()
    p.starmap(mix, mix_arg)
    p.join()
    p.close()
```  
The example will generate data with same N and different m then save them to files as adjacency list. It is just an example.  
Note: do not run this code if not needed, it will take a lot of time and memory! And it did not work in notebooks since Pool is not supported.

The degree distribution can be get by counting the degrees of all the nodes. `G.degree` gives an view of `{node: degree}` and we just need to count the degree. The frequency of each degree could be get by dived by $\sum{k_i}$ of 2 times the number of edges.

In [5]:
# for degree distribution, we need to count the occurrance
# of each degree, this could easily and fastly done by
# python stdlib
from collections import Counter

# always need the network data structure
import networkx as nx

# bokeh is a nice plot lib which plot pretty graphs and easy
# to use, I prefer it than matplotlib
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

# networkx depends on matplotlib and require us to import
# it explicitly
from matplotlib import pyplot as plt

output_notebook()

Loading BokehJS ...

We still need a function to get the degree distribution of a graph.

In [42]:
# for degree distribution, we need to count the occurrance
# of each degree, this could easily and fastly done by
# python stdlib
from collections import Counter
# C-array like data structure supported by numpy
import numpy as np

def k_distri(G):
    """
    Find the degree distribution of a graph G

    Parameters
    ----------
    G : networkx.Graph
        the targeted graph we want to find degree distribution

    Returns
    -------
    None.

    """

    # degree data
    degree = np.array([degree for nodes, degree in G.degree])
    degree.sort()
    total = degree.size

    # count the degree occurance
    degree, occurance = zip(*Counter(degree).items())

    degree,occurance, frequencies = np.array(degree),\
        np.array(occurance), np.array(occurance) / total
    return (degree, occurance, frequencies)

# an example
G = nx.complete_graph(100)
# complete graph of 100 nodes, all degree would be 99
# k=99, o=100, f=1
k, o, f = k_distri(G)
print(k)
print(o)
print(f)

[99]
[100]
[1.]


Now we will start to do the data analysing work!

```python
import implementation as imp

N = tuple([100_000] * 7)  # This is the largest scale I can get with memory
                          # on my laptop
m = (2, 4, 8, 16, 32, 64, 128)  # m from the a small number to big enough
                                # to fill my memory

args = zip(N, m, m)

for i in args:
    imp.pref(*i)
```

Then get their degree distribution

In [43]:
from degree_distribution import k_distri
import networkx as nx
import numpy as np


G=[]
raw_degree = []

for m in (2, 4, 8, 16, 32, 64, 128):
    path = ' '.join(('.\\DATA\\PrefGraph with N = 100000, m =', str(m)))
    g = nx.read_adjlist(path)
    degree = np.array([d for n, d in g.degree])
    raw_degree.append(degree)
    d = np.array([d for n, d in g.degree])
    d, o, f= k_distri(g)
    G.append((m,d,f))

Data visualisation:

In [44]:
# bokeh is a nice plot lib which plot pretty graphs and easy
# to use, I prefer it than matplotlib
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.layouts import gridplot
from logbin import logbin


output_notebook()

# the theoretical probability
@np.vectorize
def p(k, m):
    p = 2 * m**2 / k**3
    return p

x = np.linspace(100, 1_000)


p1 = figure(title='N=100000 m=2', x_axis_label='degree ln(k)', y_axis_label='frequency ln(f)',
            x_axis_type='log', y_axis_type='log')
p2 = figure(title='N=100000 m=4', x_axis_label='degree ln(k)', y_axis_label='frequency ln(f)',
            x_axis_type='log', y_axis_type='log')
p3 = figure(title='N=100000 m=8', x_axis_label='degree ln(k)', y_axis_label='frequency ln(f)',
            x_axis_type='log', y_axis_type='log')
p4 = figure(title='N=100000 m=16', x_axis_label='degree ln(k)', y_axis_label='frequency ln(f)',
            x_axis_type='log', y_axis_type='log')
p5 = figure(title='N=100000 m=32', x_axis_label='degree ln(k)', y_axis_label='frequency ln(f)',
            x_axis_type='log', y_axis_type='log')
p6 = figure(title='N=100000 m=64', x_axis_label='degree ln(k)', y_axis_label='frequency ln(f)',
            x_axis_type='log', y_axis_type='log')
p7 = figure(title='N=100000 m=128', x_axis_label='degree ln(k)', y_axis_label='frequency ln(f)',
            x_axis_type='log', y_axis_type='log')

p1.circle(G[0][1], G[0][2], legend_label="m = {}".format(G[0][0]), size=5, color='red')
p1.line(*logbin(raw_degree[0],1.25), legend_label='logbin', line_color='mediumslateblue')
p1.line(x, p(x, 2), legend_label='theory', line_color='darkred', line_dash='dashed')

p2.circle(G[1][1], G[1][2], legend_label="m = {}".format(G[1][0]), size=5, color='green')
p2.line(*logbin(raw_degree[1],1.25), legend_label='logbin', line_color='mediumslateblue')
p2.line(x, p(x, 4), legend_label='theory', line_color='darkred', line_dash='dashed')

p3.circle(G[2][1], G[2][2], legend_label="m = {}".format(G[2][0]), size=5, color='blue')
p3.line(*logbin(raw_degree[2],1.25), legend_label='logbin', line_color='mediumslateblue')
p3.line(x, p(x, 8), legend_label='theory', line_color='darkred', line_dash='dashed')

p4.circle(G[3][1], G[3][2], legend_label="m = {}".format(G[3][0]), size=5, color='yellow')
p4.line(*logbin(raw_degree[3],1.25), legend_label='logbin', line_color='mediumslateblue')
p4.line(x, p(x, 16), legend_label='theory', line_color='darkred', line_dash='dashed')

p5.circle(G[4][1], G[4][2], legend_label="m = {}".format(G[4][0]), size=5, color='purple')
p5.line(*logbin(raw_degree[4],1.25), legend_label='logbin', line_color='mediumslateblue')
p5.line(x, p(x, 32), legend_label='theory', line_color='darkred', line_dash='dashed')

p6.circle(G[5][1], G[5][2], legend_label="m = {}".format(G[5][0]), size=5, color='grey')
p6.line(*logbin(raw_degree[5],1.25), legend_label='logbin', line_color='mediumslateblue')
p6.line(x, p(x, 64), legend_label='theory', line_color='darkred', line_dash='dashed')

p7.circle(G[6][1], G[6][2], legend_label="m = {}".format(G[6][0]), size=5, color='brown')
p7.line(*logbin(raw_degree[6],1.25), legend_label='logbin', line_color='mediumslateblue')
p7.line(x, p(x, 128), legend_label='theory', line_color='darkred', line_dash='dashed')


grid = gridplot([[p1, None, p2],[p3,p4,p5],[p6,None,p7]], plot_width=280, plot_height=280)
show(grid)

Loading BokehJS ...

In [45]:
p1.circle(G[0][1], G[0][2], legend_label="m = {}".format(G[0][0]), size=5, color='red')
p1.line(*logbin(raw_degree[0],1.2), legend_label='logbin', line_color='mediumslateblue')
p1.circle(G[1][1], G[1][2], legend_label="m = {}".format(G[1][0]), size=5, color='green')
p1.line(*logbin(raw_degree[1],1.2), legend_label='logbin', line_color='mediumslateblue')
p1.circle(G[2][1], G[2][2], legend_label="m = {}".format(G[2][0]), size=5, color='blue')
p1.line(*logbin(raw_degree[2],1.2), legend_label='logbin', line_color='mediumslateblue')
p1.circle(G[3][1], G[3][2], legend_label="m = {}".format(G[3][0]), size=5, color='yellow')
p1.line(*logbin(raw_degree[3],1.2), legend_label='logbin', line_color='mediumslateblue')
p1.circle(G[4][1], G[4][2], legend_label="m = {}".format(G[4][0]), size=5, color='purple')
p1.line(*logbin(raw_degree[4],1.2), legend_label='logbin', line_color='mediumslateblue')
p1.circle(G[5][1], G[5][2], legend_label="m = {}".format(G[5][0]), size=5, color='grey')
p1.line(*logbin(raw_degree[5],1.2), legend_label='logbin', line_color='mediumslateblue')
p1.circle(G[6][1], G[6][2], legend_label="m = {}".format(G[6][0]), size=5, color='brown')
p1.line(*logbin(raw_degree[6],1.2), legend_label='logbin', line_color='mediumslateblue')
p1.plot_width = 900
p1.plot_height = 900
show(p1)

The logbinning method gives an distribution and we are not sure how good it is. we will employ $\chi^2$ test to see if it obeys the theoretical distribution.

In [46]:
from scipy.stats import chisquare
from numpy import corrcoef

x, y = logbin(raw_degree[5])
x = np.array(x)
y = np.array(y)

# theoretical value
t = p(x, 64)

test = chisquare(y, t)
print(test)

correlation_matrix = np.corrcoef(y, t)
correlation_xy = correlation_matrix[0,1]
r_squared = correlation_xy**2
print(r_squared)

Power_divergenceResult(statistic=0.0559908971155411, pvalue=1.0)
0.9991469045479388


This gives p value 1 !! This might not true because the log binning and theoretical value can hardly been no difference. I guess the floating point operation error induced this. The log binning fit the theoretical moddel well enough. We can check how the system size (with fixed m) affect result.

In [54]:
G=[]
raw_degree = []

for N in (100, 1_000, 10_000, 100_000, 1_000_000):
    path = '.\\DATA\\PrefGraph with N = {}, m = 16'.format(str(N))
    g = nx.read_adjlist(path)
    degree = np.array([d for n, d in g.degree])
    raw_degree.append(degree)
    d = np.array([d for n, d in g.degree])
    d, o, f= k_distri(g)
    G.append((16,d,f))
    
p1 = figure(title='N=100 m=16', x_axis_label='degree ln(k)', y_axis_label='frequency ln(f)',
            x_axis_type='log', y_axis_type='log')
p2 = figure(title='N=1000 m=16', x_axis_label='degree ln(k)', y_axis_label='frequency ln(f)',
            x_axis_type='log', y_axis_type='log')
p3 = figure(title='N=10000 m=16', x_axis_label='degree ln(k)', y_axis_label='frequency ln(f)',
            x_axis_type='log', y_axis_type='log')
p4 = figure(title='N=100000 m=16', x_axis_label='degree ln(k)', y_axis_label='frequency ln(f)',
            x_axis_type='log', y_axis_type='log')
p5 = figure(title='N=1000000 m=16', x_axis_label='degree ln(k)', y_axis_label='frequency ln(f)',
            x_axis_type='log', y_axis_type='log')


p1.circle(G[0][1], G[0][2], legend_label="m = {}".format(G[0][0]), size=5, color='red')
p1.line(*logbin(raw_degree[0],1.25), legend_label='logbin', line_color='mediumslateblue')

p2.circle(G[1][1], G[1][2], legend_label="m = {}".format(G[1][0]), size=5, color='green')
p2.line(*logbin(raw_degree[1],1.25), legend_label='logbin', line_color='mediumslateblue')

p3.circle(G[2][1], G[2][2], legend_label="m = {}".format(G[2][0]), size=5, color='blue')
p3.line(*logbin(raw_degree[2],1.25), legend_label='logbin', line_color='mediumslateblue')

p4.circle(G[3][1], G[3][2], legend_label="m = {}".format(G[3][0]), size=5, color='yellow')
p4.line(*logbin(raw_degree[3],1.25), legend_label='logbin', line_color='mediumslateblue')

p5.circle(G[4][1], G[4][2], legend_label="m = {}".format(G[4][0]), size=5, color='purple')
p5.line(*logbin(raw_degree[4],1.25), legend_label='logbin', line_color='mediumslateblue')


grid = gridplot([[p1, p2, p3],[p4, p5, None]], plot_width=280, plot_height=280)
show(grid)

In [65]:
dc = figure(title='Data collapse', x_axis_label='degree ln(k/k_{cutoff})', y_axis_label='frequency ln(p/p_{\infty})',
            x_axis_type='log', y_axis_type='log')

color = ('red', 'green', 'blue', 'grey', 'black')

for i in range(5):
    x, y = logbin(raw_degree[i],1.25)
    t = p(x, m)    
    L = raw_degree[i].size
    N = x.size
    m = G[i][0]
    k_cutoff = N**0.5

    X = x/k_cutoff
    Y = y*t
    dc.line(X, Y, legend_label='N={} m={}'.format(L,m), line_color=color[i])

show(dc)

In [10]:
G=[]
raw_degree = []

for m in (2, 4, 8, 16, 32, 64, 128):
    path = '.\\DATA\\RanGraph with N = 100000, m = {}'.format(str(m))
    g = nx.read_adjlist(path)
    degree = np.array([d for n, d in g.degree])
    raw_degree.append(degree)
    d, o, f= k_distri(g)
    G.append((m,d,f))

r1 = figure(title='N=100000 m=2', x_axis_label='degree k', y_axis_label='frequency f',
            x_axis_type='log', y_axis_type='log')


@np.vectorize
def pr(k, m):
    p = 1/(m+1) * (m/(m+1))**(k-m)
    return p


x1, y1 = logbin(raw_degree[0], 1.2)
r1.circle(G[0][1], G[0][2], legend_label="m = {}".format(G[0][0]), size=5, color='red')
r1.line(x1, y1, legend_label='logbin', line_color='mediumslateblue')
r1.line(x1, pr(x1, G[0][0]), legend_label='theory', line_color='darkred', line_dash='dashed')


x2, y2 = logbin(raw_degree[1], 1.2)
r1.circle(G[1][1], G[1][2], legend_label="m = {}".format(G[1][0]), size=5, color='green')
r1.line(x2, y2, legend_label='logbin', line_color='mediumslateblue')
r1.line(x2, pr(x2, G[1][0]), legend_label='theory', line_color='darkred', line_dash='dashed')

x3, y3 = logbin(raw_degree[2], 1.2)
r1.circle(G[2][1], G[2][2], legend_label="m = {}".format(G[2][0]), size=5, color='yellow')
r1.line(x3, y3, legend_label='logbin', line_color='mediumslateblue')
r1.line(x3, pr(x3, G[2][0]), legend_label='theory', line_color='darkred', line_dash='dashed')

x4, y4 = logbin(raw_degree[3], 1.2)
r1.circle(G[3][1], G[3][2], legend_label="m = {}".format(G[3][0]), size=5, color='purple')
r1.line(x4, y4, legend_label='logbin', line_color='mediumslateblue')
r1.line(x4, pr(x4, G[3][0]), legend_label='theory', line_color='darkred', line_dash='dashed')

x5, y5 = logbin(raw_degree[4], 1.2)
r1.circle(G[4][1], G[4][2], legend_label="m = {}".format(G[4][0]), size=5, color='grey')
r1.line(x5, y5, legend_label='logbin', line_color='mediumslateblue')
r1.line(x5, pr(x5, G[4][0]), legend_label='theory', line_color='darkred', line_dash='dashed')

x6, y6 = logbin(raw_degree[5], 1.2)
r1.circle(G[5][1], G[5][2], legend_label="m = {}".format(G[5][0]), size=5, color='brown')
r1.line(x6, y6, legend_label='logbin', line_color='mediumslateblue')
r1.line(x6, pr(x6, G[5][0]), legend_label='theory', line_color='darkred', line_dash='dashed')

x7, y7 = logbin(raw_degree[6], 1.2)
r1.circle(G[6][1], G[6][2], legend_label="m = {}".format(G[6][0]), size=5, color='blue')
r1.line(x7, y7, legend_label='logbin', line_color='mediumslateblue')
r1.line(x7, pr(x7, G[6][0]), legend_label='theory', line_color='darkred', line_dash='dashed')

show(r1)

Loading BokehJS ...

In [13]:
x, y = logbin(raw_degree[5])
x = np.array(x)
y = np.array(y)

from scipy import stats as st

# theoretical value
t = pr(x, G[5][0])

test = st.chisquare(y, t)
print(test)

correlation_matrix = np.corrcoef(y, t)
correlation_xy = correlation_matrix[0,1]
r_squared = correlation_xy**2
print(r_squared)

Power_divergenceResult(statistic=0.005543413008973741, pvalue=1.0)
0.99801388844966


In [21]:
for i in range(7):
    print(G[i][1][-1])
    
    def kc(m, N=100000):
        k = m - math.log(N)/(math.log(m) - math.log(m+1))
        return k
    
    print(kc(G[i][0]))
    print('**************')

31
30.394367936337858
**************
53
55.59425579258086
**************
93
105.74689029545513
**************
175
205.9051098385431
**************
316
406.1405554263808
**************
558
806.5688176715055
**************
1042
1607.4034559892852
**************


Mixed preferential:

In [22]:
G = []
raw_degree = []


def mixg(path):
    g = nx.read_adjlist(path)
    degree = np.array([d for n, d in g.degree])
    raw_degree.append(degree)
    d, o, f= k_distri(g)
    G.append((d,f))
    
    
for i in range(4):
    path = '.\\DATA\\MixGraph with N=100000, m=4, q={}_3'.format(str(i))
    mixg(path)

In [25]:
from bokeh.layouts import gridplot

mg1 = figure(title='N=100000 m=4', x_axis_label='degree k', y_axis_label='frequency f',
            x_axis_type='log', y_axis_type='log')
mg2 = figure(title='N=100000 m=4', x_axis_label='degree k', y_axis_label='frequency f',
            x_axis_type='log', y_axis_type='log')
mg3 = figure(title='N=100000 m=4', x_axis_label='degree k', y_axis_label='frequency f',
            x_axis_type='log', y_axis_type='log')
mg4 = figure(title='N=100000 m=4', x_axis_label='degree k', y_axis_label='frequency f',
            x_axis_type='log', y_axis_type='log')


mg1.circle(G[0][0], G[0][1], legend_label="q = 0", size=5, color='red')
mg1.line(*logbin(raw_degree[0],1.25), legend_label='logbin', line_color='mediumslateblue')

mg2.circle(G[1][0], G[1][1], legend_label="q = 1/3", size=5, color='green')
mg2.line(*logbin(raw_degree[1],1.25), legend_label='logbin', line_color='mediumslateblue')

mg3.circle(G[2][0], G[2][1], legend_label="q = 2/3", size=5, color='blue')
mg3.line(*logbin(raw_degree[2],1.25), legend_label='logbin', line_color='mediumslateblue')

mg4.circle(G[3][0], G[3][1], legend_label="q = 1", size=5, color='yellow')
mg4.line(*logbin(raw_degree[3],1.25), legend_label='logbin', line_color='mediumslateblue')


grid = gridplot([[mg1, mg2],[mg3, mg4]], plot_width=480, plot_height=480)
show(grid)